In [ ]:
from __future__ import print_function, division
from builtins import range, input
#future verisyonu güncellemek için;
# sudo pip install -U future

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from datetime import datetime
from sortedcontainers import SortedList

#İlk yapmamız gereken şey datayı yüklemek. Datayı yüklerken önce verilerin var olup olmadığını kontrol etmemiz gerekiyor.
#Bunuda exists komutuyla yapıyoruz.

import os
if not os.path.exists('user2movie.json') or \
   not os.path.exists('movie2user.json') or \
   not os.path.exists('usermovie2rating.json') or \
   not os.path.exists('usermovie2rating_test.json'):
   import preprocess2dict

#Bir önceki kodda oluşturduğumuz tüm sözlükleri yüklüyoruz.
with open('user2movie.json', 'rb') as f:
  user2movie = pickle.load(f)

with open('movie2user.json', 'rb') as f:
  movie2user = pickle.load(f)

with open('usermovie2rating.json', 'rb') as f:
  usermovie2rating = pickle.load(f)

with open('usermovie2rating_test.json', 'rb') as f:
  usermovie2rating_test = pickle.load(f)


N = np.max(list(user2movie.keys())) + 1
# test seti, eğitim setinde veri bulunmayan filmler içerebilir
m1 = np.max(list(movie2user.keys()))
m2 = np.max([m for (u, m), r in usermovie2rating_test.items()])
M = max(m1, m2) + 1
print("N:", N, "M:", M)

if N > 10000:
  print("N =", N, "devam etmek istediğinizden emin misiniz?")
  print("Eğer varsa bu satırları yorumlayın")
  exit()

K = 25 # dikkate almak istediğimiz komşu sayısı
limit = 5 # Kullanıcıların arasındaki ortak filmlerin en az 5 olması limiti.
neighbors = [] # komşuları bu listede saklayacağız.
averages = [] # daha sonra kullanmak üzere her kullanıcının ortalama puanı
deviations = [] # daha sonra kullanmak üzere her kullanıcının sapması
for i in range(N):
  # i kullanıcısına en yakın 25 kullanıcıyı bulan döngü
  movies_i = user2movie[i]
  movies_i_set = set(movies_i)

  # Ortalamayı ve sapmayı hesaplıyoruz.
  ratings_i = { movie:usermovie2rating[(i, movie)] for movie in movies_i }
  avg_i = np.mean(list(ratings_i.values()))
  dev_i = { movie:(rating - avg_i) for movie, rating in ratings_i.items() }
  dev_i_values = np.array(list(dev_i.values()))
  sigma_i = np.sqrt(dev_i_values.dot(dev_i_values))

  # Artık sapmaları(deviation_i) ve sigma_i'ı daha sonra kullanmak için saklayabiliriz.

  averages.append(avg_i)
  deviations.append(dev_i)

  sl = SortedList()
  for j in range(N):
    # i'yi dahil etmiyoruz.
    if j != i:
      movies_j = user2movie[j]
      movies_j_set = set(movies_j)
      common_movies = (movies_i_set & movies_j_set) # intersection
      if len(common_movies) > limit:
        # i kullanıcısında yaptığımız tüm işlemleri j kullanıcısı içinde yapıyoruz.
        ratings_j = { movie:usermovie2rating[(j, movie)] for movie in movies_j }
        avg_j = np.mean(list(ratings_j.values()))
        dev_j = { movie:(rating - avg_j) for movie, rating in ratings_j.items() }
        dev_j_values = np.array(list(dev_j.values()))
        sigma_j = np.sqrt(dev_j_values.dot(dev_j_values))

        # Korelasyonu hesaplıyoruz.
        #Numerator yani pay, iki kullanıcının ortak filmlere göre sapmalarının çarpımının toplamıdır.
        numerator = sum(dev_i[m]*dev_j[m] for m in common_movies)
        #Payda, bize w_ij'i veren Sigma'nın çarpımıdır.
        w_ij = numerator / (sigma_i * sigma_j)

        #Sıralı liste otomatik olarak artan şeklinde sıralar.Ancak, Peerson korelasyonu için, daha yüksek bir sayı, 
        #daha fazla korelasyon anlamına gelir.Dolayısıyla, en ilişkili kullanıcının listenin başında olması için,
        #aslında ağırlığın negatifini saklayacağız.Bu noktada, sıralanan listenin boyutunu, tutmak istediğimiz
        #komşu sayısını aşıp aşmadığını da kontrol edebiliriz. 
        #Eğer aşıyorsa listedeki son girişi sileriz.Ve bunu yaparak, listenin boyutu K veya daha küçük kalmasını sağlarız.
        sl.add((-w_ij, j))
        if len(sl) > K:
          del sl[-1]

  # komşuları saklıyoruz.
  neighbors.append(sl)

  if i % 1 == 0:
    print(i)


# komşuları kullanarak eğitim ve test mean squared error'u hesaplıyoruz

def predict(i, m):
  # sapmaların ağırlıklı toplamını hesaplıyoruz
  numerator = 0
  denominator = 0
  for neg_w, j in neighbors[i]:
    # Ağırlıkları sıralı listemizde negatif ile çarparak tutmuştuk bu sayede en ilişkili filmler en üstte yer alıyordu.
    # Bu yüzden negatif ile çarparak pozitifi elde ediyoruz.
    try:
      numerator += -neg_w * deviations[j][m]
      denominator += abs(neg_w)
    except KeyError:
# Bir şeyin sözlükte olup olmadığını anlamak için ona bakmamız gerekir ki bu da zaman alır. 
# Dolayısıyla, bu kontrolü yaparsak, aslında filme iki kez bakarız, bir kez orada olup olmadığını görmek için ve 
# bir kez de değerini almak için.
# Bunun yerine, öğe yoksa kodun KeyError fırlatmasına izin verebilir ve hatanın gerçekten
#herhangi bir soruna neden olmaması için pass ile geçebiliriz.
      pass

  if denominator == 0:
    prediction = averages[i]
  else:
    prediction = numerator / denominator + averages[i]
  prediction = min(5, prediction)
  prediction = max(0.5, prediction) # min rating is 0.5
  return prediction

#Artık tahminlerimizi yapmak için prediction fonksiyonumuz hazır.


#Öyleyse listeler oluşturarak başlayalım, tahminlerimizi ve hedeflerimizi saklayalım. 
#Sonra her bir derecelendirmeyi gözden geçiriyoruz.
train_predictions = []
train_targets = []
for (i, m), target in usermovie2rating.items():
  # # m filmi için i kullanıcısının tamhinini hesaplıyoruz.
  prediction = predict(i, m)

  # Tahminleri ve hedefleri listelere atıyoruz.
  train_predictions.append(prediction)
  train_targets.append(target)

#Sırada aynı işlemleri test setleri içinde yapıyoruz.
test_predictions = []
test_targets = []
for (i, m), target in usermovie2rating_test.items():
  prediction = predict(i, m)

  test_predictions.append(prediction)
  test_targets.append(target)


#Sonra, bu kare farklarını hesaplıyoruz ve sonra ortalama kare hatasını elde etmek için bu dizinin ortalamasını alıyoruz.
def mse(p, t):
  p = np.array(p)
  t = np.array(t)
  return np.mean((p - t)**2)

print('train mse:', mse(train_predictions, train_targets))
print('test mse:', mse(test_predictions, test_targets))


